In [35]:
import pandas as pd
import argparse
import random
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score

import torch

# how to using huggingface tokenizer(bert) 
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification

# using monologg/kobert tokenizer and model
from kobert_transformers.tokenization_kobert import KoBertTokenizer
from transformers import BertModel

# huggingface Trainer
from transformers import Trainer
from transformers import TrainingArguments

#Cutomizer encoder
from bert_dataset import TokenCollator
from bert_dataset import TokenDataset


from transformers.data.data_collator import DataCollatorForTokenClassification
from datasets import load_metric

import torch
from torch.utils.data import Dataset


In [36]:
def BIO_tagging(tok_sentence,ne):
  result=['O' for x in range(len(tok_sentence))]
  ne_no=len(ne.keys())
  if ne_no > 0:
    for idx in range(1,ne_no+1):
      ne_dict=ne[idx]
      isbegin=True
      for word_idx, word in enumerate(tok_sentence):
        if '##' in word:
          word=word.replace('##','')
        if '▁' in word:
          word=word.replace('_','')
        if word in ne_dict['form']:
          if isbegin:
            result[word_idx]=str(ne_dict['label'][:2])+'_B'
            isbegin=False
            continue
          elif isbegin==False & (('_B' in result[word_idx-1]) or '_I' in result[word_idx-1]):
            result[word_idx]=str(ne_dict['label'][:2])+'_I'
            continue
  return result

In [53]:
class TokenDataset(Dataset):

    def __init__(self, texts, labels) -> None: #리스트의 리스트 형태의 데이터 입력
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]
# 딕셔너리 값으로 리스트를 반환
        return {
            'text': text,
            'label': label,
        }

In [54]:
def get_datasets(texts, labels, valid_ratio=.2):

    # define unique labels
    unique_labels = ['O', 'PS_B', 'PS_I', 'LC_B', 'QT_B', 'QT_I', 'CV_B', 'CV_I', 'LC_I', 'OG_B', 'OG_I', 'DT_B', 'AM_B', 'DT_I', 'EV_B', 'EV_I', 'TM_B', 'TM_I', 'FD_B', 'FD_I', 'AM_I', 'AF_B', 'AF_I', 'TI_B', 'TI_I', 'PT_B', 'PT_I', 'TR_B', 'TR_I', 'MT_B', 'MT_I']

    label_to_index = {}
    index_to_label = {}
    for i, label in enumerate(unique_labels):
        label_to_index[label] = i
        index_to_label[i] = label

    # Convert label text to integer value
    tags = []
    for i in range(len(labels)):                                
      tags.append(list(map(label_to_index.get, labels[i]))) 

    # Shuffle before split into train and validation set.
    shuffled = list(zip(texts, tags))
    random.shuffle(shuffled)
    texts = [e[0] for e in shuffled]
    labels = [e[1] for e in shuffled]
    idx = int(len(texts) * (1 - valid_ratio))

    train_dataset = TokenDataset(texts[:idx], labels[:idx])
    valid_dataset = TokenDataset(texts[idx:], labels[idx:])

    return train_dataset, valid_dataset, index_to_label

In [ ]:
def tokenize_and_align_labels(example):
    tokenized_input = tokenizer(example["text"], truncation=True, is_split_into_words=True)

    word_ids = tokenized_input.word_ids(batch_index=0)
    previous_word_idx = None
    labels = []
    for word_idx in word_ids: 
      if word_idx is None:
        labels.append(-100)
      elif word_idx != previous_word_idx: 
        labels.append(example['label'][word_idx])
      else:
        labels.append(-100)
      previous_word_idx = word_idx

    tokenized_input["labels"] = labels
    return tokenized_input 

In [38]:
df = pd.read_csv('/content/drive/MyDrive/NER/data/train.tsv', sep='\t')

In [39]:
df

,sentence,label,source,sentence_class
0,늪 속으로 깊이 빠져드는 그 자신이다.,{},N,Out
1,절에 다니는 분들은 또 그거 가잖아,{},S,Out
2,에~ 노무현 대통령이 하신 말씀 중에,"{1: {'form': '노무현', 'label': 'PS_NAME', 'begin...",S,PS
3,뚜껑을 덮어 주시면,{},S,Out
4,미국 동부 명문 사립고인 쿠싱아카데미(Cushing Academy)의 짐 트레이시(...,"{1: {'form': '미국', 'label': 'LCP_COUNTRY', 'be...",N,TM
...,...,...,...,...
624431,어쩌면 그거를 조금 더 빨리 발견할 수 있지 않나?,{},S,Out
624432,몰라,{},S,Out
624433,네.,{},S,Out
624434,음.,{},S,Out


In [40]:
!pip show transformers


Name: transformers
Version: 4.18.0
Summary: State-of-the-art Natural Language Processing for TensorFlow 2.0 and PyTorch
Home-page: https://github.com/huggingface/transformers
Author: Thomas Wolf, Lysandre Debut, Victor Sanh, Julien Chaumond, Sam Shleifer, Patrick von Platen, Sylvain Gugger, Suraj Patil, Stas Bekman, Google AI Language Team Authors, Open AI team Authors, Facebook AI Authors, Carnegie Mellon University Authors
Author-email: thomas@huggingface.co
License: Apache
Location: /usr/local/lib/python3.7/dist-packages
Requires: filelock, pyyaml, numpy, tokenizers, huggingface-hub, packaging, sacremoses, tqdm, regex, requests, importlib-metadata
Required-by: kobert-transformers


In [41]:
sentences=df['sentence']
labels=df['label'].apply(lambda x:eval(x))

In [42]:
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
model = AutoModelForTokenClassification.from_pretrained('klue/bert-base',num_labels=32)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the mo

In [50]:
sentence_tok=[]
for sentence in sentences:
  if len(sentence) < 512:
    sentence_tok.append(tokenizer.tokenize(sentence))
  else:
    sentence_tok.append(tokenizer.tokenize(sentence,max_length=len(sentence),truncation=True))
sentence_tok[0]

['늪', '속', '##으로', '깊이', '빠져', '##드', '##는', '그', '자신', '##이다', '.']

In [51]:
bio_sentence=[]
for tok_list,label in zip(sentence_tok,labels):
  try:
    bio_sentence+=[BIO_tagging(tok_list,label)]
  except Exception as e:
    print(e)
    print(tok_list)
    print(label)
    break  

In [52]:
print(sentence_tok[4])
print(bio_sentence[4])

['미국', '동부', '명문', '사립', '##고', '##인', '쿠', '##싱', '##아', '##카', '##데미', '(', 'C', '##ush', '##ing', 'Ac', '##ade', '##my', ')', '의', '짐', '트레이', '##시', '(', 'Tr', '##acy', ')', '교장', '##이', '방한', '##했', '##다', '.']
['LC_B', 'TM_B', 'O', 'O', 'O', 'O', 'OG_B', 'OG_I', 'OG_I', 'OG_I', 'OG_I', 'O', 'OG_B', 'OG_I', 'OG_I', 'OG_I', 'OG_I', 'OG_I', 'O', 'O', 'PS_B', 'PS_I', 'PS_I', 'O', 'PS_B', 'PS_I', 'O', 'CV_B', 'PS_I', 'O', 'O', 'O', 'O']


In [ ]:
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
model = BertModel.from_pretrained('monologg/kobert',num_labels=32)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [33]:
sentence_tok=[]
for sentence in sentences:
  if len(sentence) < 512:
    sentence_tok.append(tokenizer.tokenize(sentence))
  else:
    sentence_tok.append(tokenizer.tokenize(sentence,max_length=len(sentence),truncation=True))
sentence_tok[0]

Keyword arguments {'max_length': 536, 'truncation': True} not recognized.
Keyword arguments {'max_length': 1326, 'truncation': True} not recognized.
Keyword arguments {'max_length': 580, 'truncation': True} not recognized.
Keyword arguments {'max_length': 638, 'truncation': True} not recognized.


['▁', '늪', '▁속', '으로', '▁깊이', '▁빠져', '드는', '▁그', '▁자신이', '다', '.']

In [34]:
bio_sentence=[]
for tok_list,label in zip(sentence_tok,labels):
  try:
    bio_sentence+=[BIO_tagging(tok_list,label)]
  except Exception as e:
    print(e)
    print(tok_list)
    print(label)
    break  

In [49]:
print(sentence_tok[4])
print(bio_sentence[4])

['미국', '동부', '명문', '사립', '##고', '##인', '쿠', '##싱', '##아', '##카', '##데미', '(', 'C', '##ush', '##ing', 'Ac', '##ade', '##my', ')', '의', '짐', '트레이', '##시', '(', 'Tr', '##acy', ')', '교장', '##이', '방한', '##했', '##다', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'OG_B', 'OG_I', 'O', 'OG_B', 'OG_I', 'OG_I', 'OG_I', 'O', 'PS_B', 'OG_I', 'OG_I', 'OG_I', 'PS_I', 'O', 'O', 'O', 'O', 'PS_B', 'PS_I', 'O', 'PS_I', 'PS_I', 'PS_I', 'PS_I', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [55]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sentence_tok, bio_sentence, test_size=0.2)

In [ ]:
train_dataset, valid_dataset, index_to_label = get_datasets(X_train, y_train, valid_ratio=.2)